# HANKA AI - Fine-Tuning Data Analysis

This notebook analyzes the training data for HANKA AI fine-tuning. It checks data quality, distribution, and sufficiency to ensure the best results for the fine-tuned model.

In [ ]:
# Import Libraries
import json
import pandas as pd
import matplotlib.pyplot as plt

try:
    import seaborn as sns
except ImportError:
    sns = None
    print("seaborn not installed, some plots will be simplified")

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
# Load and Parse Data
# Note: Upload your 'training_data.jsonl' or 'hanka_finetune_dataset.jsonl' file to the Colab environment first.
file_path = 'hanka_finetune_dataset.jsonl' 

records = []
errors = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                record = json.loads(line.strip())
                records.append(record)
            except json.JSONDecodeError as e:
                errors.append(f"Line {line_num}: {str(e)}")
    print(f"Successfully loaded {len(records)} records.")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please upload the dataset.")
    # Create dummy data for demonstration if file not found
    print("Creating dummy data for demonstration...")
    records = [
        {"messages": [{"role": "user", "content": "siparişler"}, {"role": "assistant", "content": "{\"sql\": \"SELECT * FROM siparisler\"}"}], "metadata": {"firma_id": 1, "timestamp": "2023-10-27 10:00:00"}},
        {"messages": [{"role": "user", "content": "müşteriler"}, {"role": "assistant", "content": "{\"sql\": \"SELECT * FROM musteri\"}"}], "metadata": {"firma_id": 1, "timestamp": "2023-10-27 11:00:00"}},
        {"messages": [{"role": "user", "content": "stok durumu"}, {"role": "assistant", "content": "{\"sql\": \"SELECT * FROM stok\"}"}], "metadata": {"firma_id": 2, "timestamp": "2023-10-28 09:00:00"}}
    ] * 20

In [ ]:
# Data Structure Analysis
df = pd.DataFrame(records)

# Helper to extract message content
def get_content(messages: list, role: str) -> str | None:
    for m in messages:
        if m['role'] == role:
            return m['content']
    return None

df['user_content'] = df['messages'].apply(lambda x: get_content(x, 'user'))
df['assistant_content'] = df['messages'].apply(lambda x: get_content(x, 'assistant'))
df['system_content'] = df['messages'].apply(lambda x: get_content(x, 'system'))

print("Dataset Sample:")
display(df[['user_content', 'assistant_content']].head())

print(f"\nTotal Records: {len(df)}")
print(f"Missing User Content: {df['user_content'].isna().sum()}")
print(f"Missing Assistant Content: {df['assistant_content'].isna().sum()}")

In [ ]:
# Token Count Estimation (Approximate)
# OpenAI uses tiktoken, but for a rough estimate we can use character count / 4
def estimate_tokens(text: str) -> float:
    if not text: return 0
    return len(str(text)) / 4

df['user_tokens'] = df['user_content'].apply(estimate_tokens)
df['assistant_tokens'] = df['assistant_content'].apply(estimate_tokens)
df['system_tokens'] = df['system_content'].apply(estimate_tokens)
df['total_tokens'] = df['user_tokens'] + df['assistant_tokens'] + df['system_tokens']

total_dataset_tokens = df['total_tokens'].sum()
estimated_cost_gpt35 = (total_dataset_tokens / 1000) * 0.0080 # Example rate for GPT-3.5 Turbo Fine-tuning

print(f"Average Tokens per Example: {df['total_tokens'].mean():.2f}")
print(f"Total Dataset Tokens (Approx): {total_dataset_tokens:,.0f}")
print(f"Estimated Training Cost (1 Epoch, GPT-3.5): ${estimated_cost_gpt35:.4f}")

# Plot Token Distribution
plt.figure(figsize=(10, 6))
if sns:
    sns.histplot(df['total_tokens'], bins=30, kde=True)
else:
    plt.hist(df['total_tokens'], bins=30)
plt.title('Token Distribution per Example')
plt.xlabel('Token Count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# SQL Query Analysis
import re

def extract_sql(content):
    try:
        # Content is usually a JSON string like {"sql": "SELECT ...", "explanation": "..."}
        data = json.loads(content)
        return data.get('sql', '')
    except:
        return ''

df['sql_query'] = df['assistant_content'].apply(extract_sql)

# Extract Table Names (Simple Regex)
def extract_tables(sql):
    # Matches FROM table or JOIN table
    # Very basic regex, might need refinement for complex queries
    tables = re.findall(r'FROM\s+`?(\w+)`?|JOIN\s+`?(\w+)`?', sql, re.IGNORECASE)
    # Flatten list of tuples and filter empty strings
    return [t for group in tables for t in group if t]

all_tables = []
for sql in df['sql_query']:
    all_tables.extend(extract_tables(sql))

table_counts = pd.Series(all_tables).value_counts()

print("Top 10 Most Accessed Tables:")
print(table_counts.head(10))

# Plot Table Usage
plt.figure(figsize=(12, 6))
table_counts.head(15).plot(kind='bar')
plt.title('Top 15 Most Accessed Tables in Training Data')
plt.xlabel('Table Name')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# System Prompt Analysis
unique_system_prompts = df['system_content'].unique()
print(f"Unique System Prompts: {len(unique_system_prompts)}")

if len(unique_system_prompts) < 10:
    for i, prompt in enumerate(unique_system_prompts):
        print(f"Prompt {i+1}: {prompt[:100]}...")
else:
    print("Top 5 System Prompts:")
    print(df['system_content'].value_counts().head(5))

# Conclusion
print("\n" + "="*50)
print("ANALYSIS CONCLUSION")
print("="*50)
if len(df) >= 50:
    print("✅ Dataset size is sufficient for initial fine-tuning.")
else:
    print("⚠️ Dataset size is small. Consider collecting more examples (aim for 50-100+).")

if df['total_tokens'].mean() < 2000:
    print("✅ Token counts are within reasonable limits.")
else:
    print("⚠️ Some examples are very long. Check for context window limits.")

print("\nReady to upload to OpenAI!")